In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')




# 하이퍼파라미터 설정
batch_size = 200
learning_rate = 0.002
epoch = 10

# MNIST 데이터셋 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = dset.MNIST(root="./", train=True, transform=transform, download=True)
test_data = dset.MNIST(root="./", train=False, transform=transform, download=True)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)

# CNN 모델 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.model(x)

# 모델, 손실 함수, 옵티마이저 설정
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


# 학습 진행

i=0
for epoch in range(epoch):
  for data, target in train_loader:
    data = data.to(device)
    target = target.to(device)

    optimizer.zero_grad()
    output=model(data)
    loss=criterion(output,target)
    loss.backward()
    optimizer.step()


    if i%1000 == 0:
      print('train step : {},\t loss : {:.3f}'.format(i,loss.item()))
    i +=1



# 테스트 진행
model.eval()
correct = 0

for data, target in test_loader:
  data = data.to(device)
  target = target.to(device)

  output = model(data)
  prediction = output.data.max(1)[1]
  correct += prediction.eq(target.data).sum()


print('accuracy : {:.2f}%'.format(100*correct/len(test_loader.dataset)))

train step : 0,	 loss : 2.307
train step : 1000,	 loss : 0.045
train step : 2000,	 loss : 0.012
accuracy : 99.35%
